In [1]:
from pathlib import Path
import sys

from peewee import (
    SqliteDatabase,
)

sys.path.append(str(Path("../src").resolve().as_posix()))
from utils.database import (
    Organism,
    Sequence,
    TMInfo,
    Annotation,
)

db_path = Path("../data/tmvis.db")
db = SqliteDatabase(db_path.resolve().as_posix())
db.bind([Organism, Sequence, TMInfo, Annotation], bind_backrefs=False, bind_refs=False)

In [2]:
from peewee import OperationalError


def get_existing_indexes(model):
    """Fetch existing indexes for a given model."""
    table_name = model._meta.table_name
    existing_indexes = set()
    cursor = db.execute_sql(f'PRAGMA index_list("{table_name}");')
    for row in cursor.fetchall():
        existing_indexes.add(row[1])
    return existing_indexes


def get_model_indexes(model):
    """Get all indexes for the model including field indexes and meta indexes."""
    indexes = []

    # Field indexes
    for field in model._meta.fields.values():
        if field.index or field.unique:
            index_name = f"{model._meta.table_name}_{field.column_name}"
            indexes.append(([field.column_name], field.unique, index_name))

    # Meta indexes
    for index_fields, is_unique in model._meta.indexes:
        index_name = (
            "_".join(index_fields) if isinstance(index_fields, tuple) else index_fields
        )
        indexes.append(
            (index_fields, is_unique, f"{model._meta.table_name}_{index_name}")
        )

    return indexes


def create_missing_indexes(model):
    """Create indexes defined in the model but missing in the database."""
    existing_indexes = get_existing_indexes(model)
    missing_indexes = get_model_indexes(model)

    for index_fields, is_unique, index_name in missing_indexes:
        if index_name not in existing_indexes:
            if isinstance(index_fields, str):
                index_fields = [index_fields]
            fields = ", ".join([f'"{field}"' for field in index_fields])
            unique = "UNIQUE" if is_unique else ""
            sql = f'CREATE {unique} INDEX IF NOT EXISTS "{index_name}" ON "{model._meta.table_name}" ({fields});'
            try:
                db.execute_sql(sql)
                print(f"Created index: {index_name}")
            except OperationalError as e:
                print(f"Error creating index {index_name}: {e}")


def ensure_indexes():
    models = [Organism, Sequence, TMInfo, Annotation]
    for model in models:
        create_missing_indexes(model)


with db:
    ensure_indexes()

Created index: organism_super_kingdom_clade
Created index: tminfo_has_alpha_helix_has_beta_strand_has_signal
Created index: annotation_sequence_start_end


In [3]:
with db:
    db.execute_sql("REINDEX")

In [ ]:
with db:
    db.execute_sql("VACUUM;")

# Execution finished without errors.
# Result: query executed successfully. Took 29678842ms
# At line 1:
# VACUUM;

OperationalError: cannot VACUUM from within a transaction

In [5]:
with db:
    db.execute_sql("ANALYZE;")